In [2]:
%pylab inline
## -*- coding: utf-8 -*-

Populating the interactive namespace from numpy and matplotlib


In [341]:
import re
import json
import time
import copy
import xml.etree.ElementTree as ET

In [4]:
tropnames = {'etnakhta': u'\u0591', 'segol': u'\u0592', 'shalshelet': u'\u0593', 'katan': u'\u0594', 'gadol': u'\u0595',
             'tipkha': u'\u0596', 'revii': u'\u0597', 'tsinnorit': u'\u0598', 'pashta': u'\u0599', 'yetiv': u'\u059a', 'tevir': u'\u059b',
             'geresh': u'\u059c', 'gereshmukdam': u'\u059d', 'gershayim': u'\u059e', 'karnepara': u'\u059f', 'telishagedola': u'\u05a0',
             'pazer': u'\u05a1', 'munakh': u'\u05a3', 'mapakh': u'\u05a4', 'merkha': u'\u05a5',
             'merkhakfula': u'\u05a6', 'darga': u'\u05a7', 'kadma': u'\u05a8', 'telishaketana': u'\u05a9', 'yerakhbenyomo': u'\u05aa',
             'sofpasuk': u'\u05c3', 'zarka': u'\u05ae'}

# per wikipedia: Note that both marks have been wrongly named by Unicode.[5][6] Zarqa/tsinnor corresponds to Unicode
# "Hebrew accent zinor", code point U+05AE (where "zinor" is a misspelled form for tsinnor), while tsinnorit maps to
# "Hebrew accent zarqa", code point U+0598. 

tropchars = {tropnames[t]: t for t in tropnames}

## Parse

In [427]:
sfarim = ['bereshit', 'shmot', 'vayikra', 'bmidbar', 'dvarim']

tropstrings = {}
for sefer in sfarim:
    tropstrings[sefer] = {}
    tree = ET.parse(sefer + '.xml')
    root = tree.getroot() 
    prakim = root.findall('.//c')
    for perek in prakim:
        pereknum = int(perek.attrib['n'])
        if pereknum not in tropstrings[sefer]: tropstrings[sefer][pereknum] = {}
        psukim = perek.findall('v')
        for pasuk in psukim:
            pasuknum = int(pasuk.attrib['n'])
            if pasuknum not in tropstrings[sefer][pereknum]:
                # row = {name: 0 for name in tropnames.keys()}
                # row['sefer'] = sefer
                # row['pasuk'] = pasuknum
                # row['perek'] = pereknum
                tropstrings[sefer][pereknum][pasuknum] = ''
#             words = pasuk.findall('w') # TODO: also look for <q> tags for qtiv
            words = [w for w in pasuk if w.tag=='w' or w.tag=='q']
            wordslist = map(lambda w: list(w.text), words)
            troplist = map(lambda w: filter(lambda c: c in tropchars, w), wordslist)
            troplist = filter(lambda w: len(w) > 0, troplist)
            # there's probably a better way to do this, but I can't think of it
            flattroplist = []
            for w in troplist:
                for c in w:
                    flattroplist.append(c)
            tropstrings[sefer][pereknum][pasuknum] += ''.join(flattroplist)

## Build the tree (and the seqsearch helper)

In [643]:
zeros = {s: {pe: {pa: 0 for pa in tropstrings[s][pe]} for pe in tropstrings[s]} for s in tropstrings}
lengths = {s: {pe: {pa: len(tropstrings[s][pe][pa]) for pa in tropstrings[s][pe]} for pe in tropstrings[s]} for s in tropstrings}

In [645]:
def seqsearch(seq, s):
#     print len(sources)
    seqcount = 0
    
    seqlist = ','.join(map(lambda c: tropchars[c], list(seq)))
    if seqlist not in sequencecountsbypasuk:
        sequencecountsbypasuk[seqlist] = {}
        
    if s != []:
        newsources = []
        for source in s:
#             print sources
            found = re.findall(seq, tropstrings[source[0]][source[1]][source[2]])
            seqcount += len(found)
            if len(found) > 0:
                newsources.append(source)
                
                if source[0] not in sequencecountsbypasuk[seqlist]: sequencecountsbypasuk[seqlist][source[0]] = {}
                if source[1] not in sequencecountsbypasuk[seqlist][source[0]]: sequencecountsbypasuk[seqlist][source[0]][source[1]] = {}
#                 if source[2] not in sequencecountsbypasuk[seq][source[0]][source[1]]: sequencecountsbypasuk[seq][source[0]][source[1]][source[2]] = {}
                sequencecountsbypasuk[seqlist][source[0]][source[1]][source[2]] = len(found)
        if len(newsources) > 0: s = newsources
    else:
        for sefer in tropstrings:
            for perek in tropstrings[sefer]:
                for pasuk in tropstrings[sefer][perek]:
                    found = re.findall(seq, tropstrings[sefer][perek][pasuk])
                    seqcount += len(found)
                    if len(found) > 0:
                        s.append((sefer, perek, pasuk))
                        
                        if sefer not in sequencecountsbypasuk[seqlist]: sequencecountsbypasuk[seqlist][sefer] = {}
                        if perek not in sequencecountsbypasuk[seqlist][sefer]: sequencecountsbypasuk[seqlist][sefer][perek] = {}
#                         if pasuk not in sequencecountsbypasuk[seq][sefer][perek][pasuk]: sequencecountsbypasuk[seq][sefer][perek][pasuk] = {}
                        sequencecountsbypasuk[seqlist][sefer][perek][pasuk] = len(found)
    return (seqcount, s)

In [429]:
map(lambda t: tropchars[t], tropstrings['dvarim'][16][18])

['munakh',
 'revii',
 'pashta',
 'katan',
 'kadma',
 'darga',
 'tevir',
 'merkha',
 'tipkha',
 'etnakhta',
 'merkha',
 'tipkha',
 'sofpasuk']

In [582]:
seqsearch(tropnames['merkha']+tropnames['merkha'], [])

(1, [('dvarim', 5, 8)])

In [396]:
def getcounts(seq, sources):
    parent = []
    for trop in tropnames:
        thisseq = seq+tropnames[trop]
#         seqlist = map(lambda c: tropchars[c], list(thisseq)) # for printing
#         print "Searching for " + str(seqlist) + " in " + str(len(sources)) + " psukim"
        seqcount, localsources = seqsearch(thisseq, copy.copy(sources))
#         print "Found " + str(seqcount) + " in " + str(len(sources)) + " psukim"
        if seqcount > 0:
            parent.append({'name': tropchars[tropnames[trop]], 'char': tropnames[trop], 'count': seqcount});
            parent[len(parent)-1]['children'] = getcounts(thisseq, localsources)
#         sources = [] # if you finish all of the trop, go back to looking in all. stupid way to make this happen.
    return parent

In [725]:
# this is going to be an object keyed on sequence,sefer,perek,pasuk. it will be aggregated by perek later
sequencecountsbypasuk = {}

start = time.time()
tree = getcounts('', [])
print time.time() - start

326.187469959


In [395]:
tree['munakh']['children']['revii']['children']['pashta']['children']['katan']['count']

92

In [333]:
sum([len(tropstrings[s]) for s in tropstrings])

192

### Output the json

In [431]:
with open('sequencetree-d3format.json', 'wb') as outfile:
    json.dump(tree, outfile)

## Sources

In [451]:
import pandas as pd

In [466]:
sfarim = ['bereshit', 'shmot', 'vayikra', 'bmidbar', 'dvarim']

In [726]:
flatcounts = [{'seq': seq, 'sefer': sefer, 'perek': perek, 'pasuk': pasuk, 'count': sequencecountsbypasuk[seq][sefer][perek][pasuk]} for seq in sequencecountsbypasuk for sefer in sequencecountsbypasuk[seq] for perek in sequencecountsbypasuk[seq][sefer] for pasuk in sequencecountsbypasuk[seq][sefer][perek]]
df = pd.DataFrame(flatcounts)
df.set_index(['seq', 'sefer', 'perek', 'pasuk'], inplace=True)
df = df.reindex(sfarim, level=1)

In [661]:
flatlengths = [{'sefer': sefer, 'perek': perek, 'pasuk': pasuk, 'length': lengths[sefer][perek][pasuk]} for sefer in lengths for perek in lengths[sefer] for pasuk in lengths[sefer][perek]]
lengthsdf = pd.DataFrame(flatlengths)
lengthsdf.set_index(['sefer', 'perek', 'pasuk'], inplace=True)
lengthsdf = lengthsdf.reindex(sfarim, level=0)

In [711]:
pereklengths = lengthsdf.groupby(level=['sefer','perek']).aggregate(sum)

In [727]:
byperek = df.groupby(level=['seq','sefer','perek']).aggregate(sum)

In [665]:
# byperek['totaltrop'] = pereklengths

In [652]:
# byperek['norm'] = byperek['count']/byperek['totaltrop']

In [713]:
bypereklist = []
for row in byperek.loc['revii,merkha'].iterrows():
    bypereklist.append({'index': row[0][0] + ',' + str(row[0][1]), 'count': row[1]['count'], 'norm': float(row[1]['count'])/pereklengths.loc[row[0][0],row[0][1]]['length']})
bypereklist

[{'count': 2, 'index': 'bereshit,1', 'norm': 0.005434782608695652},
 {'count': 1, 'index': 'bereshit,6', 'norm': 0.0038314176245210726},
 {'count': 1, 'index': 'bereshit,15', 'norm': 0.004405286343612335},
 {'count': 1, 'index': 'bereshit,16', 'norm': 0.005050505050505051},
 {'count': 2, 'index': 'bereshit,17', 'norm': 0.006230529595015576},
 {'count': 1, 'index': 'bereshit,18', 'norm': 0.0024630541871921183},
 {'count': 1, 'index': 'bereshit,19', 'norm': 0.002079002079002079},
 {'count': 1, 'index': 'bereshit,20', 'norm': 0.004166666666666667},
 {'count': 1, 'index': 'bereshit,22', 'norm': 0.003289473684210526},
 {'count': 1, 'index': 'bereshit,23', 'norm': 0.004366812227074236},
 {'count': 1, 'index': 'bereshit,26', 'norm': 0.002386634844868735},
 {'count': 1, 'index': 'bereshit,29', 'norm': 0.0024330900243309003},
 {'count': 1, 'index': 'bereshit,30', 'norm': 0.0020325203252032522},
 {'count': 1, 'index': 'bereshit,31', 'norm': 0.0015037593984962407},
 {'count': 1, 'index': 'bereshi

In [578]:
# with open('byperek_munakhrevii.json', 'wb') as outfile:
#     json.dump(bypereklist, outfile)

In [654]:
byperek.loc['revii,munakh']

count  totaltrop      norm
sefer    perek                            
bereshit 11         1         18  0.055556
         18         1         16  0.062500
         19         1         19  0.052632
         20         1         20  0.050000
         26         2         35  0.057143
         27         2         39  0.051282
         28         1         21  0.047619
         29         1         14  0.071429
         30         1         16  0.062500
         32         1         13  0.076923
         34         1         18  0.055556
         35         1         17  0.058824
         38         1         16  0.062500
         42         3         50  0.060000
         43         2         49  0.040816
         45         1         19  0.052632
         47         2         37  0.054054
         50         2         41  0.048780
shmot    3          1         17  0.058824
         4          1         19  0.052632
         6          1         18  0.055556
         7          1         23  0.043478
         10         2         30  0.066667
         11         1         20  0.050000
         12         2         34  0.058824
         13         2         45  0.044444
         16         1         25  0.040000
         20         1         29  0.034483
         26         2         33  0.060606
         28         1         20  0.050000
...               ...        ...       ...
bmidbar  13         1         26  0.038462
         14         1         17  0.058824
         15         1         22  0.045455
         16         3         43  0.069767
         17         3         65  0.046154
         19         1         21  0.047619
         24         1         15  0.066667
         26         1         17  0.058824
         27         1         16  0.062500
         28         1         19  0.052632
         31         1         14  0.071429
         32         1         20  0.050000
         35         1         24  0.041667
dvarim   1          1         19  0.052632
         4          1         22  0.045455
         5          3         82  0.036585
         9          3         56  0.053571
         11         1         21  0.047619
         13         1         29  0.034483
         14         2         43  0.046512
         15         2         42  0.047619
         18         1         19  0.052632
         22         2         34  0.058824
         24         1         16  0.062500
         27         1         23  0.043478
         28         3         64  0.046875
         29         1         17  0.058824
         30         2         26  0.076923
         31         3         58  0.051724
         32         1         19  0.052632

[76 rows x 3 columns]

I really wish I understood why byperek.index.levels[0], sequencecountsbypasuk.keys(), and the 'seq' element of flatcounts are all different. Suspicious.

In [728]:
for seq in byperek.index.levels[0]:
    bypereklist = []
    for row in byperek.loc[seq].iterrows():
        bypereklist.append({'index': row[0][0] + ',' + str(row[0][1]), 'count': row[1]['count'], 'norm': float(row[1]['count'])/pereklengths.loc[row[0][0],row[0][1]]['length']})
    with open('treeviz/byperek/byperek_' + seq.replace(',', '') + '.json', 'wb') as outfile:
        json.dump(bypereklist, outfile)